## Cardiff NLP Hackathon 2025 - Starter Code

Welcome to Cardiff NLP's second hackathon! Below is some code to get started on the AMPLYFI API and look at some data.

====================

Note: the API is a real time resource so extra points to projects that can treat it as a continual data stream rather than a one-off data source!

Another thing to note about this is that it will affect Amplyfi's servers if you download a silly amount of data. We ask that you only request 100 results per request, but if you have the data you need, try to download it or store it as a variable rather than requesting the exact same data over and over again.

In [299]:
# Import some libraries

import requests
import json
import nltk
import re
import pandas as pd
from collections import defaultdict
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go







from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Amplyfi have provided some limits and explanations of what you can query the API for below:

`query_text` anything

`result_size` <=100

`include_highlights` boolean (if True, you get sentences matching keyphrases in the query)

`include_smart_tags` boolean (if True, you get back metadata from our "smart" market intel classifiers - signals and industries)

`ai_answer` can only accept "basic", this will take the 5 most relevant docs and answer the query_text based on them




In [300]:
# API endpoint from the newly deployed service

API_URL = "https://zfgp45ih7i.execute-api.eu-west-1.amazonaws.com/sandbox/api/search"
API_KEY = "ZR38746G38B7RB46GBER"

headers = {
    "Content-Type": "application/json",
    "x-api-key": API_KEY
}

query_text = "Hong Kong"

# Edit the below to get different data
payload = {
  "query_text": query_text,
  "result_size": 100,
  "include_highlights":True,
  "timerange": "180d"
}

response = requests.post(API_URL, headers=headers, data=json.dumps(payload))
json_response = response.json()

json_response


{'results': [{'id': 'DL-4c86a7943f2c15bfa36f4ca6e389b565',
   'title': 'Explore Hong Kong SAR to Indonesia routes with Singapore Airlines',
   'url': 'https://www.singaporeair.com/en-sg/flights-from-hong-kong-sar-to-indonesia',
   'summary': 'Book the best flight from Hong Kong SAR to Indonesia From To *Fares displayed have been collected within the last 48 hours and may no longer be available at the time of booking. Quoted fares include taxes, fees, and surcharges.',
   'score': 655.1164,
   'timestamp': '2025-02-18T04:01:57+00:00',
   'highlights': ['Book the best flight from Hong Kong SAR to Indonesia\nFrom\nTo\n*Fares displayed have been collected within the last 48 hours and may no longer be available at the time of booking. Quoted fares include taxes, fees, and surcharges.',
    'Find a great way to fly from Hong Kong SAR to Indonesia\nFind a great way to fly from Hong Kong SAR to Indonesia\nFrom\nTo\nFare Type\nDates\nPrice\nFromHong Kong SAR (HKG)\nToDenpasar Bali (DPS)\nRound-

In [301]:
grouped_results = defaultdict(list)

for item in json_response["results"]:
  if not item["timestamp"]:
    continue
  date = item["timestamp"][0:7]  # Extract date part
  grouped_results[date].append(item)

# Convert defaultdict to a regular dictionary
grouped_results = dict(grouped_results)

# Print grouped results
print(json.dumps(grouped_results, indent=4))

{
    "2025-02": [
        {
            "id": "DL-4c86a7943f2c15bfa36f4ca6e389b565",
            "title": "Explore Hong Kong SAR to Indonesia routes with Singapore Airlines",
            "url": "https://www.singaporeair.com/en-sg/flights-from-hong-kong-sar-to-indonesia",
            "summary": "Book the best flight from Hong Kong SAR to Indonesia From To *Fares displayed have been collected within the last 48 hours and may no longer be available at the time of booking. Quoted fares include taxes, fees, and surcharges.",
            "score": 655.1164,
            "timestamp": "2025-02-18T04:01:57+00:00",
            "highlights": [
                "Book the best flight from Hong Kong SAR to Indonesia\nFrom\nTo\n*Fares displayed have been collected within the last 48 hours and may no longer be available at the time of booking. Quoted fares include taxes, fees, and surcharges.",
                "Find a great way to fly from Hong Kong SAR to Indonesia\nFind a great way to fly from Hong Ko

In [302]:
df1 = pd.json_normalize(grouped_results['2025-01'])
df2 = pd.json_normalize(grouped_results['2025-02'])
df3 = pd.json_normalize(grouped_results['2025-03'])
df4 = pd.json_normalize(grouped_results['2025-04'])
df5 = pd.json_normalize(grouped_results['2025-05'])
df6 = pd.json_normalize(grouped_results['2025-06'])
#df7 = pd.json_normalize(grouped_results['2025-06-10'])




## Example Sentiment Analysis

In [303]:
## Clean data

def clean_text(text):
    """
    - Convert to lowercase
    - Remove URLs
    - Remove punctuation / non-alpha
    - Collapse multiple spaces
    """
    if not isinstance(text, str):
        return ""
    # Remove URLs (very basic)
    text = re.sub(r"http\S+|www\.\S+", "", text)
    # Lowercase
    text = text.lower()
    # Keep only letters and spaces
    text = re.sub(r"[^a-z\s]", " ", text)
    # Collapse multiple spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

df1['clean_summary'] = df1['summary'].apply(clean_text)
df2['clean_summary'] = df2['summary'].apply(clean_text)
df3['clean_summary'] = df3['summary'].apply(clean_text)
df4['clean_summary'] = df4['summary'].apply(clean_text)
df5['clean_summary'] = df5['summary'].apply(clean_text)
df6['clean_summary'] = df6['summary'].apply(clean_text)
#df7['clean_summary'] = df7['summary'].apply(clean_text)


In [304]:
## Sentiment analysis example

sia = SentimentIntensityAnalyzer()

def get_sentiment_scores(text):
    """
    Returns a dict with these keys:
       - neg: negative sentiment score
       - neu: neutral score
       - pos: positive score
       - compound: normalized, weighted composite (-1 to +1)
    """
    return sia.polarity_scores(text)

# Apply to each summary
df1['sentiment'] = df1['clean_summary'].apply(get_sentiment_scores)
df2['sentiment'] = df2['clean_summary'].apply(get_sentiment_scores)
df3['sentiment'] = df3['clean_summary'].apply(get_sentiment_scores)
df4['sentiment'] = df4['clean_summary'].apply(get_sentiment_scores)
df5['sentiment'] = df5['clean_summary'].apply(get_sentiment_scores)
df6['sentiment'] = df6['clean_summary'].apply(get_sentiment_scores)
#df7['sentiment'] = df7['clean_summary'].apply(get_sentiment_scores)

# Split into separate columns if you like
df1['sent_neg'] = df1['sentiment'].apply(lambda d: d['neg'])
df1['sent_neu'] = df1['sentiment'].apply(lambda d: d['neu'])
df1['sent_pos'] = df1['sentiment'].apply(lambda d: d['pos'])
df1['sent_compound'] = df1['sentiment'].apply(lambda d: d['compound'])

df2['sent_neg'] = df2['sentiment'].apply(lambda d: d['neg'])
df2['sent_neu'] = df2['sentiment'].apply(lambda d: d['neu'])
df2['sent_pos'] = df2['sentiment'].apply(lambda d: d['pos'])
df2['sent_compound'] = df2['sentiment'].apply(lambda d: d['compound'])

df3['sent_neg'] = df3['sentiment'].apply(lambda d: d['neg'])
df3['sent_neu'] = df3['sentiment'].apply(lambda d: d['neu'])
df3['sent_pos'] = df3['sentiment'].apply(lambda d: d['pos'])
df3['sent_compound'] = df3['sentiment'].apply(lambda d: d['compound'])

df4['sent_neg'] = df4['sentiment'].apply(lambda d: d['neg'])
df4['sent_neu'] = df4['sentiment'].apply(lambda d: d['neu'])
df4['sent_pos'] = df4['sentiment'].apply(lambda d: d['pos'])
df4['sent_compound'] = df4['sentiment'].apply(lambda d: d['compound'])

df5['sent_neg'] = df5['sentiment'].apply(lambda d: d['neg'])
df5['sent_neu'] = df5['sentiment'].apply(lambda d: d['neu'])
df5['sent_pos'] = df5['sentiment'].apply(lambda d: d['pos'])
df5['sent_compound'] = df5['sentiment'].apply(lambda d: d['compound'])

df6['sent_neg'] = df6['sentiment'].apply(lambda d: d['neg'])
df6['sent_neu'] = df6['sentiment'].apply(lambda d: d['neu'])
df6['sent_pos'] = df6['sentiment'].apply(lambda d: d['pos'])
df6['sent_compound'] = df6['sentiment'].apply(lambda d: d['compound'])

"""
df7['sent_neg'] = df7['sentiment'].apply(lambda d: d['neg'])
df7['sent_neu'] = df7['sentiment'].apply(lambda d: d['neu'])
df7['sent_pos'] = df7['sentiment'].apply(lambda d: d['pos'])
df7['sent_compound'] = df7['sentiment'].apply(lambda d: d['compound'])
"""



# Quick look at top 5 compound scores
print(df1[['timestamp', 'clean_summary', 'sent_compound']].sort_values(by='sent_compound', ascending=False).head())
print(df1[['timestamp', 'clean_summary', 'sent_compound']].sort_values(by='sent_compound').head())


                    timestamp  \
4         2025-01-24T14:07:57   
11        2025-01-24T00:00:00   
8         2025-01-24T12:39:57   
7   2025-01-24T11:33:18+00:00   
9   2025-01-24T14:55:00+00:00   

                                        clean_summary  sent_compound  
4   secretary of the hong kong special administrat...         0.9382  
11  secretary of the hong kong special administrat...         0.9382  
8   financial secretary of the hong kong special a...         0.9382  
7   financial secretary of the hong kong special a...         0.9382  
9   financial secretary of the hong kong special a...         0.9382  
                    timestamp  \
15  2025-01-20T15:10:39+00:00   
16                 2025-01-15   
1   2025-01-21T06:14:10+00:00   
14  2025-01-31T11:00:00+00:00   
12  2025-01-16T11:00:00+00:00   

                                        clean_summary  sent_compound  
15  mcdonald s officially opened in hong kong on j...         0.0000  
16  hong kong offers a high qualit

In [305]:
def todate(ts):
  return datetime.strptime(ts, "%Y-%m").strftime("%d-%b")

dfs = [df1, df2, df3, df4, df5, df6]

x_data = [
    todate(df1['timestamp'][0][0:7]),
    todate(df2['timestamp'][0][0:7]),
    todate(df3['timestamp'][0][0:7]),
    todate(df4['timestamp'][0][0:7]),
    todate(df5['timestamp'][0][0:7]),
    todate(df6['timestamp'][0][0:7]),
    #todate(df7['timestamp'][0][0:19])
    ]
data = []
y_data = [df['sent_compound'].tolist() for df in [df1, df2, df3, df4, df5, df6]]
colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
          'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)', 'rgba(127, 96, 0, 0.5)', 'rgba(255, 99, 71, 0.5)']

# Create figure
fig = go.Figure()

# Add traces for each day
for xd, yd, cls in zip(x_data, y_data, colors):
    fig.add_trace(go.Box(
        y=yd,
        name=xd,
        boxpoints='all',  # Show all points
        jitter=0.5,  # Spread points
        whiskerwidth=0.2,
        fillcolor=cls,
        marker_size=2,
        line_width=1
    ))

# Update layout
fig.update_layout(
    title=dict(text=query_text),
    yaxis=dict(
        autorange=True,
        showgrid=True,
        zeroline=True,
        zerolinecolor='green',
        zerolinewidth=3,  # Increase thickness for better visibility
        dtick=0.2,  # Adjust tick spacing for sentiment scores
        gridcolor='rgb(255, 255, 255)',
        gridwidth=1,
    ),
    margin=dict(l=40, r=30, b=80, t=100),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
    showlegend=False
)

# Show plot
fig.show()



# Initialize lists to store results
max_scores = []
min_scores = []
max_summaries = []
min_summaries = []

# Loop through each dataframe
for i, df in enumerate(dfs, start=1):
    max_idx = df['sent_compound'].idxmax()
    min_idx = df['sent_compound'].idxmin()

    max_score = df.loc[max_idx, 'sent_compound']
    min_score = df.loc[min_idx, 'sent_compound']
    max_summary = df.loc[max_idx, 'summary']
    min_summary = df.loc[min_idx, 'summary']

    max_scores.append(max_score)
    min_scores.append(min_score)
    max_summaries.append(max_summary)
    min_summaries.append(min_summary)

    print(f"Day {i} - Most positive summary (compound = {max_score:.3f}):\n{max_summary}\n")
    print(f"Day {i} - Most negative summary (compound = {min_score:.3f}):\n{min_summary}\n")
    print("-" * 80)


Day 1 - Most positive summary (compound = 0.938):
Secretary of the Hong Kong Special Administrative Region (SAR), Paul Chan, wrapped up his participation at the World Economic Forum Annual Meeting in Davos, Switzerland (January 23) Mr Chan noted that while geopolitics may subject regional and global financial markets to greater volatility, Hong Kong boasts a robust financial system and strong buffer. He added that Hong Kong is actively embracing financial innovation, including the development of digital assets.

Day 1 - Most negative summary (compound = 0.000):
McDonald's officially opened in Hong Kong on January 26, 1975. It was brought to Hong Kong by Daniel Ng Yat Chiu () who has a doctorate in chemical engineering. Since then, having burgers and cola has become part of Hong Kong residents' life.

--------------------------------------------------------------------------------
Day 2 - Most positive summary (compound = 0.967):
The Hong Kong Trade Development Council welcomes the 2025

## Project Ideas

Feel free to use this code to start your own project, and here are some (Chat-GPT generated 😬) ideas for projects:

* Real-Time Sentiment Pulse: Visualize sentiment trends over the past 24-48 hours for any keyword.

* One-Click News Brief: Generate a 3-sentence summary of today's top articles on a given topic.

* Bias/Slant Detector: Compare headlines from multiple outlets on the same event and label their bias.

* Event Timeline Generator: Autofill a chronological list of key dates and summaries for any query.

* Breaking News Alert Bot: Push a short alert whenever article volume spikes or sentiment turns extreme.

* Multilingual Hashtag Trend Mapper: Show related hashtags and translations across different languages.

* Rumor vs. Fact Checker: Verify a user-provided statement against recent reputable sources.

* “What's Changed?” Comparator: Highlight how coverage of a topic has shifted from last month to last week.

* Geo-Mood Map: Color-code countries by average sentiment or topic intensity on a query.

* Voice-Activated News Q&A: Let users speak a question and hear back a 2–3 sentence summary of current events.

## Dashboard libraries for Python

https://shiny.posit.co/py/

https://dash.plotly.com/

https://streamlit.io/

